# How to Interact with the Team from Your Application

There are two main ways to provide feedback to the team while using your application:

1. **During a run** – Use a `UserProxyAgent` while `run()` or `run_stream()` is executing.  
2. **After a run ends** – Provide feedback as input when calling `run()` or `run_stream()` again.

## Providing Feedback During a Run

The `UserProxyAgent` is a special built-in agent that allows users to give feedback to the team while it is running.

### How to Use the `UserProxyAgent`
- Create an instance of `UserProxyAgent`.  
- Add it to the team before starting the run.  
- The team will decide when to ask for user feedback through the `UserProxyAgent`.

### When is `UserProxyAgent` Called?
- **In a RoundRobinGroupChat team** – It is called in the order it was added to the team.  
- **In a SelectorGroupChat team** – The selector prompt or function decides when the `UserProxyAgent` is called.  


In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Suggest three creative ideas for a new mobile app.")
# Use asyncio.run(...) when running in a script.
await Console(stream)


---------- user ----------
Suggest three creative ideas for a new mobile app.


---------- assistant ----------
Here are three creative ideas for a new mobile app:

1. **Local Adventure Finder**: This app encourages users to explore their local area by offering personalized adventure suggestions based on their interests, time availability, and fitness level. It could include features like geo-tagged hidden gems, themed scavenger hunts, and integration with local events. Users can earn points or rewards for completing adventures, which can be redeemed at local businesses.

2. **Mood Journey**: An interactive journaling app that allows users to document their daily emotions through text, mood tracking, and multimedia entries. The app could offer guided prompts based on users’ moods and provide insights through visual graphs and charts. Users can create a "mood map", track patterns over time, and receive tailored suggestions for improving their emotional well-being, such as activity recommendations or mindfulness exercises.

3. **Skill Swap Marketplace**: A platform 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Suggest three creative ideas for a new mobile app.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=260), metadata={}, content='Here are three creative ideas for a new mobile app:\n\n1. **Local Adventure Finder**: This app encourages users to explore their local area by offering personalized adventure suggestions based on their interests, time availability, and fitness level. It could include features like geo-tagged hidden gems, themed scavenger hunts, and integration with local events. Users can earn points or rewards for completing adventures, which can be redeemed at local businesses.\n\n2. **Mood Journey**: An interactive journaling app that allows users to document their daily emotions through text, mood tracking, and multimedia entries. The app could offer guided prompts based on users’ moods and provide insights through vis

# Providing Feedback for the Next Run  

In many cases, an application or user interacts with a team of agents in a loop:  

1. The team runs until it meets a termination condition.  
2. The application or user gives feedback.  
3. The team runs again using that feedback.  

## Why is This Useful?  
This method works well in long-running sessions where communication happens asynchronously.  

- After the team finishes a run, the application **saves the team’s state**.  
- The state is stored safely in a database or persistent storage.  
- When feedback is received, the team **resumes from where it left off** and continues running.  

This approach ensures smooth interactions between the team and the user, even across multiple sessions.  


In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client)

# Create the team setting a maximum number of turns to 1.
team = RoundRobinGroupChat([assistant], max_turns=1)

task = "Suggest three creative ideas for a new mobile app."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break


---------- user ----------
Suggest three creative ideas for a new mobile app.
---------- assistant ----------
Here are three creative ideas for a new mobile app:

1. **Virtual Time Capsule**: An app that allows users to create digital time capsules by uploading photos, videos, and messages. Users can set a future date for the capsule to open, and they can choose to share it with friends or keep it private. The app can include augmented reality features for a more immersive experience when opening the capsule in the future.

2. **Mood-Based Meal Planner**: An app that suggests recipes and meal plans based on the user’s current mood. Users can select their emotional state (happy, sad, stressed, etc.), and the app will recommend comforting or energetic meals, including shopping lists and prep instructions. It could also integrate with a mindfulness feature for cooking and enjoying meals mindfully.

3. **Eco-Adventure Finder**: A nature-focused app that connects users with local eco-friend

> #### You can see that the team stopped immediately after one agent responded.